# Phase 2-1: TabTransformer 구현 (lucidrains/tab-transformer-pytorch 기반)

검증된 TabTransformer 구현체를 활용하여 암 예후 예측 모델 구현

## 목표
- **CoxTabTransformer**: Clinical categorical + Multi-omics [measured_value, cox_coefficient] pairs
- **MethylationTabTransformer**: High-dimensional methylation data with learnable feature selection
- **End Point**: 3년 생존 여부 Binary Classification (0: 생존, 1: 3년 내 사망)

## 1. 환경 설정 및 라이브러리 임포트

In [ ]:
# 필요한 라이브러리가 설치되어 있는지 확인
!pip install tab-transformer-pytorch

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, roc_auc_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from tab_transformer_pytorch import TabTransformer
from tqdm import tqdm
import warnings
import os
import json
import pickle
from pathlib import Path
import sys

# 프로젝트 경로 추가
sys.path.append('../')
from src.models.cox_tabtransformer import CoxTabTransformer
from src.models.methylation_tabtransformer import MethylationTabTransformer
from src.utils.tabtransformer_utils import *

warnings.filterwarnings('ignore')

# 스타일 설정
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")

## 2. 기존 TabTransformer 동작 확인

In [ ]:
print("=== 기존 TabTransformer 동작 확인 ===")

# 간단한 예제로 라이브러리 테스트
test_model = TabTransformer(
    categories = (10, 5, 6),     # 3개 범주형 특성
    num_continuous = 8,          # 8개 수치형 특성
    dim = 32,
    dim_out = 1,
    depth = 3,
    heads = 4
)

# 테스트 데이터
batch_size = 16
categorical_test = torch.tensor([
    [0, 1, 2],
    [1, 2, 3],
    [2, 3, 4],
    [3, 4, 5]
]).repeat(4, 1)  # 16 samples

continuous_test = torch.randn(batch_size, 8)

# Forward pass 테스트
output = test_model(categorical_test, continuous_test)

print(f"✅ TabTransformer 테스트 완료")
print(f"   범주형 입력 shape: {categorical_test.shape}")
print(f"   수치형 입력 shape: {continuous_test.shape}")
print(f"   출력 shape: {output.shape}")
print(f"   모델 파라미터 수: {sum(p.numel() for p in test_model.parameters()):,}")

## 3. 데이터 로드 및 전처리

In [ ]:
# 데이터 경로 설정
DATA_DIR = Path('../data/processed')
RESULTS_DIR = Path('../results')
MODELS_DIR = Path('../src/models')

# 디렉토리 생성
RESULTS_DIR.mkdir(exist_ok=True, parents=True)
MODELS_DIR.mkdir(exist_ok=True, parents=True)

print("Loading data...")
# Cox 관련 데이터
cox_data = pd.read_parquet(DATA_DIR / 'integrated_table_cox.parquet')
clinical_data = pd.read_parquet(DATA_DIR / 'processed_clinical_data.parquet')

# Methylation 관련 데이터
methylation_data = pd.read_parquet(DATA_DIR / 'methylation_table.parquet')
clinical_data_methylation = pd.read_parquet(DATA_DIR / 'processed_clinical_data_for_methylation.parquet')

print(f"✅ Cox 데이터 로드: {cox_data.shape}")
print(f"✅ 임상 데이터 (Cox): {clinical_data.shape}")
print(f"✅ 메틸레이션 데이터: {methylation_data.shape}")
print(f"✅ 임상 데이터 (Methylation): {clinical_data_methylation.shape}")

## 4. Cox 계수 로드 및 생존 라벨 생성

In [ ]:
# Cox 계수 로드
try:
    cox_coefficients = load_cox_coefficients(DATA_DIR / 'cox_coefficients_lookup.pkl')
    print(f"✅ Cox 계수 로드 완료: {len(cox_coefficients)}개")
except Exception as e:
    print(f"⚠️ Cox 계수 로드 실패: {e}")
    print("샘플 Cox 계수 생성...")
    # 샘플 Cox 계수 생성 (실제로는 Cox regression으로 계산해야 함)
    cox_coefficients = {col: np.random.normal(0, 0.5) for col in cox_data.columns[:1000]}

# Cox 데이터용 생존 라벨 생성
survival_labels_cox, valid_patient_ids_cox = create_survival_labels(clinical_data, 1095)

print(f"\n=== Cox 데이터 3년 생존 예측 End Point ===")
print(f"전체 환자 수: {len(clinical_data)}")
print(f"유효한 환자 수: {len(survival_labels_cox)}")
print(f"3년 생존: {np.sum(survival_labels_cox == 0)} ({np.mean(survival_labels_cox == 0)*100:.1f}%)")
print(f"3년 내 사망: {np.sum(survival_labels_cox == 1)} ({np.mean(survival_labels_cox == 1)*100:.1f}%)")

# Methylation 데이터용 생존 라벨 생성
survival_labels_meth, valid_patient_ids_meth = create_survival_labels(clinical_data_methylation, 1095)

print(f"\n=== Methylation 데이터 3년 생존 예측 End Point ===")
print(f"전체 환자 수: {len(clinical_data_methylation)}")
print(f"유효한 환자 수: {len(survival_labels_meth)}")
print(f"3년 생존: {np.sum(survival_labels_meth == 0)} ({np.mean(survival_labels_meth == 0)*100:.1f}%)")
print(f"3년 내 사망: {np.sum(survival_labels_meth == 1)} ({np.mean(survival_labels_meth == 1)*100:.1f}%)")

## 5. Cox 데이터 전처리 (Clinical + Multi-omics)

In [ ]:
print("=== Cox 데이터 전처리 ===")

# 유효한 환자들로 데이터 필터링
cox_data_filtered = cox_data.loc[cox_data.index.intersection(valid_patient_ids_cox)]
clinical_data_filtered = clinical_data.loc[clinical_data.index.intersection(valid_patient_ids_cox)]

print(f"필터링된 Cox 데이터: {cox_data_filtered.shape}")
print(f"필터링된 임상 데이터: {clinical_data_filtered.shape}")

# Cox [측정값, 계수] 쌍 데이터 준비
cox_continuous, cox_feature_names = prepare_cox_data(
    cox_data_filtered, 
    cox_coefficients
)

# 임상 범주형 데이터 준비
clinical_categorical, vocab_sizes, encoders, clinical_feature_names = prepare_clinical_data(
    clinical_data_filtered
)

print(f"\nCox continuous shape: {cox_continuous.shape}")
print(f"Clinical categorical shape: {clinical_categorical.shape}")
print(f"Vocabulary sizes: {vocab_sizes}")
print(f"Clinical features: {clinical_feature_names}")

# 공통 환자 인덱스로 정렬
common_cox_patients = cox_data_filtered.index.tolist()
labels_dict_cox = dict(zip(valid_patient_ids_cox, survival_labels_cox))
cox_labels_aligned = np.array([labels_dict_cox[pid] for pid in common_cox_patients])

print(f"정렬된 Cox 라벨 shape: {cox_labels_aligned.shape}")
print(f"Cox 데이터 전처리 완료!")

## 6. CoxTabTransformer 모델 구현 및 테스트

In [ ]:
print("=== CoxTabTransformer 모델 테스트 ===")

# 모델 하이퍼파라미터
cox_model_config = {
    'clinical_categories': vocab_sizes,
    'num_omics_features': len(cox_feature_names),
    'dim': 64,
    'depth': 4,
    'heads': 8,
    'attn_dropout': 0.1,
    'ff_dropout': 0.1,
    'survival_hidden_dim': 256
}

print(f"모델 설정: {cox_model_config}")

# 모델 인스턴스 생성
cox_model = CoxTabTransformer(**cox_model_config)
cox_model = cox_model.to(device)

print(f"CoxTabTransformer 파라미터 수: {sum(p.numel() for p in cox_model.parameters()):,}")

# Forward pass 테스트
with torch.no_grad():
    # 작은 배치로 테스트
    test_clinical = clinical_categorical[:8].to(device)
    test_omics = cox_continuous[:8].to(device)
    
    survival_logit, representation = cox_model(test_clinical, test_omics)
    
    print(f"✅ CoxTabTransformer forward pass 성공")
    print(f"   임상 입력 shape: {test_clinical.shape}")
    print(f"   오믹스 입력 shape: {test_omics.shape}")
    print(f"   생존 logit shape: {survival_logit.shape}")
    print(f"   Representation shape: {representation.shape}")
    
    # Probability 변환
    survival_prob = torch.sigmoid(survival_logit)
    print(f"   생존 확률 범위: {survival_prob.min().item():.3f} - {survival_prob.max().item():.3f}")

## 7. 메틸레이션 데이터 전처리

In [ ]:
print("=== 메틸레이션 데이터 전처리 ===")

# 유효한 환자들로 필터링
methylation_filtered = methylation_data.loc[
    methylation_data.index.intersection(valid_patient_ids_meth)
]

print(f"필터링된 메틸레이션 데이터: {methylation_filtered.shape}")

# 메틸레이션 데이터 전처리 (variance filtering)
methylation_tensor, selected_probe_names = prepare_methylation_data(
    methylation_filtered, 
    variance_threshold=0.01
)

# 라벨 정렬
common_meth_patients = methylation_filtered.index.tolist()
labels_dict_meth = dict(zip(valid_patient_ids_meth, survival_labels_meth))
meth_labels_aligned = np.array([labels_dict_meth[pid] for pid in common_meth_patients])

print(f"전처리된 메틸레이션 tensor shape: {methylation_tensor.shape}")
print(f"선택된 probe 수: {len(selected_probe_names)}")
print(f"메틸레이션 라벨 shape: {meth_labels_aligned.shape}")
print(f"메모리 사용량 예상: {methylation_tensor.numel() * 4 / 1024**2:.1f} MB")

## 8. MethylationTabTransformer 모델 테스트

In [ ]:
print("=== MethylationTabTransformer 모델 테스트 ===")

# 모델 설정
meth_model_config = {
    'num_probes': methylation_tensor.shape[1],
    'selected_probes': 2000,  # 메모리 고려하여 조정
    'dim': 64,
    'depth': 3,
    'heads': 8,
    'attn_dropout': 0.1,
    'ff_dropout': 0.1,
    'survival_hidden_dim': 256
}

print(f"모델 설정: {meth_model_config}")

# 모델 인스턴스 생성
meth_model = MethylationTabTransformer(**meth_model_config)
meth_model = meth_model.to(device)

print(f"MethylationTabTransformer 파라미터 수: {sum(p.numel() for p in meth_model.parameters()):,}")

# Forward pass 테스트
with torch.no_grad():
    # 작은 배치로 테스트
    test_meth = methylation_tensor[:4].to(device)
    
    survival_logit, representation, selected_indices = meth_model(test_meth)
    
    print(f"✅ MethylationTabTransformer forward pass 성공")
    print(f"   메틸레이션 입력 shape: {test_meth.shape}")
    print(f"   생존 logit shape: {survival_logit.shape}")
    print(f"   Representation shape: {representation.shape}")
    print(f"   선택된 인덱스 shape: {selected_indices.shape}")
    
    # 선택된 probe 정보
    print(f"   첫 번째 샘플에서 선택된 probe 예시: {selected_indices[0][:10]}")

## 9. 데이터 분할 및 DataLoader 생성

In [ ]:
print("=== 데이터 분할 ===")

# Cox 데이터 분할
if len(cox_labels_aligned) > 100:  # 최소 데이터 수 확인
    # Clinical과 Omics 데이터를 함께 분할
    combined_cox_data = torch.cat([clinical_categorical, cox_continuous], dim=1)
    
    X_train_cox, X_val_cox, X_test_cox, y_train_cox, y_val_cox, y_test_cox = split_data_stratified(
        combined_cox_data, cox_labels_aligned, 
        test_size=0.2, val_size=0.2, random_state=42
    )
    
    # Clinical과 Omics 부분으로 다시 분리
    clinical_dim = clinical_categorical.shape[1]
    
    X_train_clinical = X_train_cox[:, :clinical_dim]
    X_train_omics = X_train_cox[:, clinical_dim:]
    X_val_clinical = X_val_cox[:, :clinical_dim]
    X_val_omics = X_val_cox[:, clinical_dim:]
    X_test_clinical = X_test_cox[:, :clinical_dim]
    X_test_omics = X_test_cox[:, clinical_dim:]
    
    print(f"Cox 데이터 분할:")
    print(f"  Train: {len(y_train_cox)} samples")
    print(f"  Val: {len(y_val_cox)} samples")
    print(f"  Test: {len(y_test_cox)} samples")
    
    # DataLoader 생성 (Cox)
    from torch.utils.data import DataLoader, TensorDataset
    
    train_dataset_cox = TensorDataset(
        X_train_clinical.long(), X_train_omics, 
        torch.tensor(y_train_cox, dtype=torch.float32)
    )
    val_dataset_cox = TensorDataset(
        X_val_clinical.long(), X_val_omics, 
        torch.tensor(y_val_cox, dtype=torch.float32)
    )
    test_dataset_cox = TensorDataset(
        X_test_clinical.long(), X_test_omics, 
        torch.tensor(y_test_cox, dtype=torch.float32)
    )
    
    train_loader_cox = DataLoader(train_dataset_cox, batch_size=32, shuffle=True)
    val_loader_cox = DataLoader(val_dataset_cox, batch_size=32, shuffle=False)
    test_loader_cox = DataLoader(test_dataset_cox, batch_size=32, shuffle=False)
    
    cox_data_ready = True
else:
    print("Cox 데이터가 부족합니다.")
    cox_data_ready = False

# 메틸레이션 데이터 분할
if len(meth_labels_aligned) > 100:
    X_train_meth, X_val_meth, X_test_meth, y_train_meth, y_val_meth, y_test_meth = split_data_stratified(
        methylation_tensor, meth_labels_aligned,
        test_size=0.2, val_size=0.2, random_state=42
    )
    
    print(f"\n메틸레이션 데이터 분할:")
    print(f"  Train: {len(y_train_meth)} samples")
    print(f"  Val: {len(y_val_meth)} samples")
    print(f"  Test: {len(y_test_meth)} samples")
    
    # DataLoader 생성 (Methylation)
    train_dataset_meth = TensorDataset(X_train_meth, torch.tensor(y_train_meth, dtype=torch.float32))
    val_dataset_meth = TensorDataset(X_val_meth, torch.tensor(y_val_meth, dtype=torch.float32))
    test_dataset_meth = TensorDataset(X_test_meth, torch.tensor(y_test_meth, dtype=torch.float32))
    
    train_loader_meth = DataLoader(train_dataset_meth, batch_size=16, shuffle=True)  # 작은 배치 사이즈
    val_loader_meth = DataLoader(val_dataset_meth, batch_size=16, shuffle=False)
    test_loader_meth = DataLoader(test_dataset_meth, batch_size=16, shuffle=False)
    
    meth_data_ready = True
else:
    print("메틸레이션 데이터가 부족합니다.")
    meth_data_ready = False

print("\n✅ 데이터 분할 및 DataLoader 생성 완료!")

## 10. 훈련 함수 구현

In [ ]:
def train_cox_tabtransformer(model, train_loader, val_loader, epochs=20, lr=1e-4):
    """
    CoxTabTransformer 훈련 함수
    """
    model = model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    criterion = nn.BCEWithLogitsLoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.7)
    
    train_losses = []
    val_losses = []
    val_aucs = []
    best_val_auc = 0
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0
        train_preds = []
        train_labels = []
        
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]")
        for clinical_batch, omics_batch, labels_batch in progress_bar:
            clinical_batch = clinical_batch.to(device)
            omics_batch = omics_batch.to(device)
            labels_batch = labels_batch.to(device)
            
            optimizer.zero_grad()
            
            # Forward pass
            logits, _ = model(clinical_batch, omics_batch)
            loss = criterion(logits.squeeze(), labels_batch)
            
            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_loss += loss.item()
            train_preds.extend(torch.sigmoid(logits).detach().cpu().numpy())
            train_labels.extend(labels_batch.cpu().numpy())
            
            progress_bar.set_postfix({'loss': loss.item():.4f})
        
        # Validation
        model.eval()
        val_loss = 0
        val_preds = []
        val_labels_list = []
        
        with torch.no_grad():
            for clinical_batch, omics_batch, labels_batch in tqdm(val_loader, desc=f"Epoch {epoch+1} [Val]"):
                clinical_batch = clinical_batch.to(device)
                omics_batch = omics_batch.to(device)
                labels_batch = labels_batch.to(device)
                
                logits, _ = model(clinical_batch, omics_batch)
                loss = criterion(logits.squeeze(), labels_batch)
                
                val_loss += loss.item()
                val_preds.extend(torch.sigmoid(logits).cpu().numpy())
                val_labels_list.extend(labels_batch.cpu().numpy())
        
        # Metrics 계산
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        
        train_auc = roc_auc_score(train_labels, train_preds)
        val_auc = roc_auc_score(val_labels_list, val_preds)
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        val_aucs.append(val_auc)
        
        print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        print(f"         Train AUC: {train_auc:.4f}, Val AUC: {val_auc:.4f}")
        
        scheduler.step(val_loss)
        
        # Best model 저장
        if val_auc > best_val_auc:
            best_val_auc = val_auc
            torch.save(model.state_dict(), 'best_cox_tabtransformer.pth')
            print(f"  ✅ Best model saved (AUC: {val_auc:.4f})")
    
    return {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'val_aucs': val_aucs,
        'best_val_auc': best_val_auc
    }

def train_methylation_tabtransformer(model, train_loader, val_loader, epochs=15, lr=5e-5):
    """
    MethylationTabTransformer 훈련 함수
    """
    model = model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    criterion = nn.BCEWithLogitsLoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=3, factor=0.7)
    
    train_losses = []
    val_losses = []
    val_aucs = []
    best_val_auc = 0
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0
        train_preds = []
        train_labels = []
        
        progress_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} [Train]")
        for meth_batch, labels_batch in progress_bar:
            meth_batch = meth_batch.to(device)
            labels_batch = labels_batch.to(device)
            
            optimizer.zero_grad()
            
            # Forward pass
            logits, _, _ = model(meth_batch)
            loss = criterion(logits.squeeze(), labels_batch)
            
            # Backward pass
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            train_loss += loss.item()
            train_preds.extend(torch.sigmoid(logits).detach().cpu().numpy())
            train_labels.extend(labels_batch.cpu().numpy())
            
            progress_bar.set_postfix({'loss': loss.item():.4f})
        
        # Validation
        model.eval()
        val_loss = 0
        val_preds = []
        val_labels_list = []
        
        with torch.no_grad():
            for meth_batch, labels_batch in tqdm(val_loader, desc=f"Epoch {epoch+1} [Val]"):
                meth_batch = meth_batch.to(device)
                labels_batch = labels_batch.to(device)
                
                logits, _, _ = model(meth_batch)
                loss = criterion(logits.squeeze(), labels_batch)
                
                val_loss += loss.item()
                val_preds.extend(torch.sigmoid(logits).cpu().numpy())
                val_labels_list.extend(labels_batch.cpu().numpy())
        
        # Metrics 계산
        train_loss /= len(train_loader)
        val_loss /= len(val_loader)
        
        train_auc = roc_auc_score(train_labels, train_preds)
        val_auc = roc_auc_score(val_labels_list, val_preds)
        
        train_losses.append(train_loss)
        val_losses.append(val_loss)
        val_aucs.append(val_auc)
        
        print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}")
        print(f"         Train AUC: {train_auc:.4f}, Val AUC: {val_auc:.4f}")
        
        scheduler.step(val_loss)
        
        # Best model 저장
        if val_auc > best_val_auc:
            best_val_auc = val_auc
            torch.save(model.state_dict(), 'best_methylation_tabtransformer.pth')
            print(f"  ✅ Best model saved (AUC: {val_auc:.4f})")
    
    return {
        'train_losses': train_losses,
        'val_losses': val_losses,
        'val_aucs': val_aucs,
        'best_val_auc': best_val_auc
    }

print("✅ 훈련 함수 정의 완료!")

## 11. 모델 훈련 실행

In [ ]:
# CoxTabTransformer 훈련
if cox_data_ready:
    print("=" * 60)
    print("CoxTabTransformer 훈련 시작...")
    print("=" * 60)
    
    cox_history = train_cox_tabtransformer(
        cox_model,
        train_loader_cox,
        val_loader_cox,
        epochs=10,  # 빠른 테스트를 위해 적은 에폭
        lr=1e-4
    )
    
    print(f"\n🏆 CoxTabTransformer 최고 성능: {cox_history['best_val_auc']:.4f}")
else:
    print("Cox 데이터가 준비되지 않았습니다.")

print("\n" + "=" * 60)

In [ ]:
# MethylationTabTransformer 훈련
if meth_data_ready:
    print("=" * 60)
    print("MethylationTabTransformer 훈련 시작...")
    print("=" * 60)
    
    meth_history = train_methylation_tabtransformer(
        meth_model,
        train_loader_meth,
        val_loader_meth,
        epochs=8,  # 빠른 테스트를 위해 적은 에폭
        lr=5e-5
    )
    
    print(f"\n🏆 MethylationTabTransformer 최고 성능: {meth_history['best_val_auc']:.4f}")
else:
    print("메틸레이션 데이터가 준비되지 않았습니다.")

print("\n" + "=" * 60)

## 12. 훈련 결과 시각화

In [ ]:
def plot_training_history(history, title):
    """훈련 과정 시각화"""
    fig, axes = plt.subplots(1, 2, figsize=(12, 4))
    
    # Loss 곡선
    axes[0].plot(history['train_losses'], label='Train Loss', marker='o')
    axes[0].plot(history['val_losses'], label='Val Loss', marker='s')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title(f'{title} - Loss Curves')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # AUC 곡선
    axes[1].plot(history['val_aucs'], label='Val AUC', marker='s', color='orange')
    axes[1].axhline(y=history['best_val_auc'], color='r', linestyle='--', 
                   label=f'Best AUC: {history["best_val_auc"]:.4f}')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('AUC')
    axes[1].set_title(f'{title} - Validation AUC')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    axes[1].set_ylim([0.5, 1.0])
    
    plt.tight_layout()
    plt.savefig(f'{title.lower().replace(" ", "_")}_training.png', dpi=100, bbox_inches='tight')
    plt.show()

# 결과 시각화
if cox_data_ready and 'cox_history' in locals():
    plot_training_history(cox_history, 'CoxTabTransformer')

if meth_data_ready and 'meth_history' in locals():
    plot_training_history(meth_history, 'MethylationTabTransformer')

## 13. 모델 평가 및 해석

In [ ]:
def evaluate_model(model, test_loader, model_name, is_cox_model=True):
    """모델 평가 함수"""
    model.eval()
    test_preds = []
    test_labels = []
    
    with torch.no_grad():
        for batch in tqdm(test_loader, desc=f"Evaluating {model_name}"):
            if is_cox_model:
                clinical_batch, omics_batch, labels_batch = batch
                clinical_batch = clinical_batch.to(device)
                omics_batch = omics_batch.to(device)
                logits, _ = model(clinical_batch, omics_batch)
            else:
                meth_batch, labels_batch = batch
                meth_batch = meth_batch.to(device)
                logits, _, _ = model(meth_batch)
            
            test_preds.extend(torch.sigmoid(logits).cpu().numpy())
            test_labels.extend(labels_batch.numpy())
    
    test_preds = np.array(test_preds).squeeze()
    test_labels = np.array(test_labels).squeeze()
    
    # 메트릭 계산
    test_auc = roc_auc_score(test_labels, test_preds)
    test_preds_binary = (test_preds > 0.5).astype(int)
    test_acc = accuracy_score(test_labels, test_preds_binary)
    
    print(f"\n=== {model_name} Test Performance ===")
    print(f"Test AUC: {test_auc:.4f}")
    print(f"Test Accuracy: {test_acc:.4f}")
    
    # Confusion Matrix
    cm = confusion_matrix(test_labels, test_preds_binary)
    print(f"\nConfusion Matrix:")
    print(f"TN: {cm[0,0]}, FP: {cm[0,1]}")
    print(f"FN: {cm[1,0]}, TP: {cm[1,1]}")
    
    # Classification Report
    print(f"\nClassification Report:")
    print(classification_report(test_labels, test_preds_binary, 
                              target_names=['Survived (0)', 'Died (1)']))
    
    return test_auc, test_acc

# 모델 평가 실행
if cox_data_ready and 'cox_history' in locals():
    # Best model 로드
    cox_model.load_state_dict(torch.load('best_cox_tabtransformer.pth'))
    cox_test_auc, cox_test_acc = evaluate_model(
        cox_model, test_loader_cox, "CoxTabTransformer", is_cox_model=True
    )

if meth_data_ready and 'meth_history' in locals():
    # Best model 로드
    meth_model.load_state_dict(torch.load('best_methylation_tabtransformer.pth'))
    meth_test_auc, meth_test_acc = evaluate_model(
        meth_model, test_loader_meth, "MethylationTabTransformer", is_cox_model=False
    )

## 14. Feature Importance 분석

In [ ]:
# 메틸레이션 모델의 선택된 probe 분석
if meth_data_ready and 'meth_history' in locals():
    print("=== 메틸레이션 Feature Importance 분석 ===")
    
    # 모델에서 중요한 probe 추출
    with torch.no_grad():
        sample_meth = methylation_tensor[:10].to(device)
        importance_scores, top_indices = meth_model.extract_feature_importance(sample_meth)
        
        print(f"상위 20개 중요한 probe:")
        top_20_indices = top_indices[:20].cpu().numpy()
        top_20_scores = importance_scores[top_20_indices].cpu().numpy()
        
        for i, (idx, score) in enumerate(zip(top_20_indices, top_20_scores)):
            probe_name = selected_probe_names[idx] if idx < len(selected_probe_names) else f"Probe_{idx}"
            print(f"{i+1:2d}. {probe_name}: {score:.4f}")
    
    # 시각화
    plt.figure(figsize=(12, 6))
    plt.bar(range(20), top_20_scores)
    plt.xlabel('Top 20 Probes (ranked by importance)')
    plt.ylabel('Importance Score')
    plt.title('Top 20 Most Important Methylation Probes')
    plt.xticks(range(20), [f'P{i+1}' for i in range(20)], rotation=45)
    plt.tight_layout()
    plt.savefig('methylation_feature_importance.png', dpi=100, bbox_inches='tight')
    plt.show()

## 15. 최종 요약 및 결과 저장

In [ ]:
# 최종 결과 요약
results_summary = {
    'experiment_info': {
        'framework': 'lucidrains/tab-transformer-pytorch',
        'endpoint': '3-year survival binary classification',
        'device': str(device)
    },
    'data_info': {
        'cox_patients': len(cox_labels_aligned) if cox_data_ready else 0,
        'methylation_patients': len(meth_labels_aligned) if meth_data_ready else 0,
        'clinical_features': len(clinical_feature_names) if cox_data_ready else 0,
        'omics_features': len(cox_feature_names) if cox_data_ready else 0,
        'methylation_probes': methylation_tensor.shape[1] if meth_data_ready else 0
    },
    'model_performance': {}
}

if cox_data_ready and 'cox_history' in locals():
    results_summary['model_performance']['cox_tabtransformer'] = {
        'best_val_auc': cox_history['best_val_auc'],
        'test_auc': cox_test_auc if 'cox_test_auc' in locals() else None,
        'test_accuracy': cox_test_acc if 'cox_test_acc' in locals() else None,
        'parameters': sum(p.numel() for p in cox_model.parameters())
    }

if meth_data_ready and 'meth_history' in locals():
    results_summary['model_performance']['methylation_tabtransformer'] = {
        'best_val_auc': meth_history['best_val_auc'],
        'test_auc': meth_test_auc if 'meth_test_auc' in locals() else None,
        'test_accuracy': meth_test_acc if 'meth_test_acc' in locals() else None,
        'parameters': sum(p.numel() for p in meth_model.parameters()),
        'selected_probes': meth_model_config['selected_probes']
    }

# 결과 저장
with open(RESULTS_DIR / 'tabtransformer_results.json', 'w') as f:
    json.dump(results_summary, f, indent=2)

print("=" * 60)
print("📊 FINAL RESULTS SUMMARY")
print("=" * 60)
print(json.dumps(results_summary, indent=2))
print("\n✅ TabTransformer 구현 및 실험 완료!")
print(f"✅ 결과 저장됨: {RESULTS_DIR / 'tabtransformer_results.json'}")

if cox_data_ready and 'cox_history' in locals():
    print(f"🎯 CoxTabTransformer 최고 성능: {cox_history['best_val_auc']:.4f} AUC")

if meth_data_ready and 'meth_history' in locals():
    print(f"🎯 MethylationTabTransformer 최고 성능: {meth_history['best_val_auc']:.4f} AUC")